# Capstone Project - The Battle of Neighborhoods

#### Salvador Roca  Guillén - IBM Professional Data Science Certification

## 1. Problem definition and data sources

A close friend of us is running a fast food business in Sacramento and, as his company is generating great profit, he has the intention to make his business grow by **expanding to San Francisco**. As he know we have studied a *comprehensive Data Science course endorsed by IBM*, he is asking us to perform an deep analysis with the objective of **determining the best zones of the city to establish the first restaurant** in the city.

To perform this analysis, we will need some information:
- **San Francisco geographical data**, which can be found in the link below. With this information, we will be able to identify the different **zones of the city** and represent the data in a visual manner by generating a **Choropleth maps**: https://data.sfgov.org/Geographic-Locations-and-Boundaries/San-Francisco-ZIP-Codes/srq6-hmpi
- **Population by ZIP code**; it is interesting to select the zones with the higher population density in order to generate a higher demand. This information can be found in the next webpage: https://www.zip-codes.com/city/ca-san-francisco.asp
- Data of all **different businesses in each zone of San Francisco**. We will filter the data to consider only the type of business that can be a competitive threat for the restaurant. The main source to obtain this information ***Foursquare*** database.

## 2. Data gathering and pre-processing

It is essential to import the libraries needed for our analysis.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import folium
import csv
import requests
from sklearn.cluster import KMeans

Next, we will import our data from the provided links.

In [2]:
!wget --quiet -O sanfran_geo.json "https://data.sfgov.org/api/geospatial/srq6-hmpi?method=export&format=GeoJSON"
!wget --quiet -O sanfran_geo.csv "https://data.sfgov.org/api/views/srq6-hmpi/rows.csv?accessType=DOWNLOAD"
!wget --quiet -O population.csv "https://drive.google.com/uc?export=download&id=1lub6B2ga28BYgoGE0ZqENjGZ26E539bW"

Let's prepare our initial dataframe, containing all ZIP codes and the associated population, and clean the data we will not need for our analysis:

In [3]:
df = pd.read_csv('population.csv')
df = df.drop(columns=["Type", "County", "Area Code(s)"])
df = df.replace(',','', regex=True)
df = df.astype({"ZIP Code": str, "Population": int})
df

,ZIP Code,Population
0,94102,31176
1,94103,2717
2,94104,406
3,94105,5846
4,94107,26599
5,94108,13768
6,94109,55984
7,94110,69333
8,94111,3713
9,94112,79407


We can see there are 27 different ZIP codes in San Francisco Area. Let's represent the population density of each zone in a Choropleth map.

In [4]:
# San Francisco latitude and longitude values
latitude = 37.77
longitude = -122.42

# create map and display it
sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=12)

sanfran_geo = r'sanfran_geo.json'
sanfran_map.choropleth(
    geo_data=sanfran_geo,
    data=df,
    columns=['ZIP Code', 'Population'],
    key_on='feature.properties.zip',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population in San Francisco'
)

sanfran_map

For our business case, it is interesting to identify which zones have more population:

In [5]:
df.nlargest(5, 'Population')

,ZIP Code,Population
9,94112,79407
7,94110,69333
16,94122,56023
6,94109,55984
12,94116,43698


In [6]:
CLIENT_ID = 'OYTC5ITWBG1R4XIWJKQ2QQYPY0BOQKTNOEO3HDYTPGIQJJ3Q' # your Foursquare ID
CLIENT_SECRET = 'JZFSM4WGITFCRASKFBKPOSJFA55RPIHDUHDVIRZ4FEDAXXO4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200
def getVenues(zip_codes):
    
    venues_list=[]
    
    for zip_code in (zip_codes):
        print(zip_code)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            zip_code,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each venue
        venues_list.append([(
            zip_code,
            v['venue']['name'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['ZIP Code',
                  'Venue',
                  'Category']
    return(nearby_venues)

sanfran_venues = getVenues(zip_codes=df['ZIP Code'])

sanfran_venues.head(10)

94102
94103
94104
94105
94107
94108
94109
94110
94111
94112
94114
94115
94116
94117
94118
94121
94122
94123
94124
94127
94129
94130
94131
94132
94133
94134
94158


,ZIP Code,Venue,Category
0,94102,The Olympic Club,Gym / Fitness Center
1,94102,SFJazz Center,Jazz Club
2,94102,Vive La Tarte,Café
3,94102,Asian Art Museum,Art Museum
4,94102,Kuma Sushi + Sake,Sushi Restaurant
5,94102,Salt & Straw,Ice Cream Shop
6,94102,Blue Bottle Coffee,Coffee Shop
7,94102,Sightglass Coffee,Coffee Shop
8,94102,Ritual Coffee Roasters,Coffee Shop
9,94102,Liholiho Yacht Club,Hawaiian Restaurant


After retrieving all the venues of each zone, we will only consider restaurants, as for our business case they are the only type of venue that can become a threat for the potential opening.

In [7]:
sanfran_venues = sanfran_venues[sanfran_venues.Category.str.contains("Restaurant")].reset_index(drop=True)
sanfran_venues.head(10)

,ZIP Code,Venue,Category
0,94102,Kuma Sushi + Sake,Sushi Restaurant
1,94102,Liholiho Yacht Club,Hawaiian Restaurant
2,94102,Bellota,Spanish Restaurant
3,94102,Domo Sushi,Sushi Restaurant
4,94102,Octavia,American Restaurant
5,94102,Daeho,Korean Restaurant
6,94102,Ryoko's Japanese Restaurant & Bar,Sushi Restaurant
7,94102,Tselogs,Filipino Restaurant
8,94102,Brenda's French Soul Food,Southern / Soul Food Restaurant
9,94102,Lers Ros Thai,Thai Restaurant


In [8]:
sanfran_venues.shape

(578, 3)

In total, we have identified **578 different restaurants in San Francisco**. Let's put together all the information by getting the number of restaurants per ZIP Code, appending the population by ZIP Code as well. After doing this, we will be able to perform our analysis on the resulting dataframe.

In [16]:
# count restaurants per ZIP Code
analysis_df = sanfran_venues.groupby('ZIP Code').count().reset_index(drop=False)

# clean columns that are not adding value and rename them to be coherent with data
analysis_df = analysis_df.drop(columns=["Category"])
analysis_df.columns = ["ZIP Code", "Restaurants Count"]

# append population per ZIP Code 
analysis_df = analysis_df.merge(df, how='outer')

analysis_df

,ZIP Code,Restaurants Count,Population
0,94102,24,31176
1,94103,17,2717
2,94104,24,406
3,94105,24,5846
4,94107,21,26599
5,94108,27,13768
6,94109,21,55984
7,94110,23,69333
8,94111,36,3713
9,94112,25,79407


## 3. Analysis: k-means Clustering

The selection of the best zones to open a restaurant will be made according to a k-means clustering, which will identify up to 5 different groups (clusters) of zone types.

In [20]:
# set number of clusters
kclusters = 5

sanfran_clustering = analysis_df.drop('ZIP Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sanfran_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_
analysis_df['Cluster Labels'] = kmeans.labels_
analysis_df

,ZIP Code,Restaurants Count,Population,Cluster Labels
0,94102,24,31176,0
1,94103,17,2717,1
2,94104,24,406,1
3,94105,24,5846,1
4,94107,21,26599,0
5,94108,27,13768,1
6,94109,21,55984,2
7,94110,23,69333,4
8,94111,36,3713,1
9,94112,25,79407,4


After performing k-means clustering, we need to see what is inside each cluster. For this, we will divide the resulting dataframe by Cluster Label.

### Cluster 1

In [22]:
cluster1 = analysis_df.loc[analysis_df['Cluster Labels'] == 0]
cluster1

,ZIP Code,Restaurants Count,Population,Cluster Labels
0,94102,24,31176,0
4,94107,21,26599,0
10,94114,17,31124,0
17,94123,21,23088,0
19,94127,23,19289,0
22,94131,18,26881,0
23,94132,8,28129,0
24,94133,29,26237,0


In [48]:
print('Restaurants Count mean: ')
rest_c1 = cluster1.loc[:,'Restaurants Count'].mean()
rest_c1

Restaurants Count mean: 


20.125

In [49]:
print('Population mean: ')
pop_c1 = cluster1.loc[:,'Population'].mean()
pop_c1

Population mean: 


26565.375

In [64]:
ratio_c1 = pop_c1 / rest_c1
ratio_c1

1320.0186335403728

### Cluster 2: low population

In [23]:
cluster2 = analysis_df.loc[analysis_df['Cluster Labels'] == 1]
cluster2

,ZIP Code,Restaurants Count,Population,Cluster Labels
1,94103,17,2717,1
2,94104,24,406,1
3,94105,24,5846,1
5,94108,27,13768,1
8,94111,36,3713,1
20,94129,8,3183,1
21,94130,1,288,1
26,94158,15,4792,1


In [50]:
print('Restaurants Count mean: ')
rest_c2 = cluster2.loc[:,'Restaurants Count'].mean()
rest_c2

Restaurants Count mean: 


19.0

In [51]:
print('Population mean: ')
pop_c2 = cluster2.loc[:,'Population'].mean()
pop_c2

Population mean: 


4339.125

In [65]:
ratio_c2 = pop_c2 / rest_c2
ratio_c2

228.375

### Cluster 3

In [24]:
cluster3 = analysis_df.loc[analysis_df['Cluster Labels'] == 2]
cluster3

,ZIP Code,Restaurants Count,Population,Cluster Labels
6,94109,21,55984,2
16,94122,35,56023,2


In [52]:
print('Restaurants Count mean: ')
rest_c3 = cluster3.loc[:,'Restaurants Count'].mean()
rest_c3

Restaurants Count mean: 


28.0

In [53]:
print('Population mean: ')
pop_c3 = cluster3.loc[:,'Population'].mean()
pop_c3

Population mean: 


56003.5

In [66]:
ratio_c3 = pop_c3 / rest_c3
ratio_c3

2000.125

### Cluster 4

In [25]:
cluster4 = analysis_df.loc[analysis_df['Cluster Labels'] == 3]
cluster4

,ZIP Code,Restaurants Count,Population,Cluster Labels
11,94115,24,33021,3
12,94116,26,43698,3
13,94117,18,39169,3
14,94118,24,38319,3
15,94121,31,41203,3
18,94124,23,33996,3
25,94134,15,40798,3


In [54]:
print('Restaurants Count mean: ')
rest_c4 = cluster4.loc[:,'Restaurants Count'].mean()
rest_c4

Restaurants Count mean: 


23.0

In [55]:
print('Population mean: ')
pop_c4 = cluster4.loc[:,'Population'].mean()
pop_c4

Population mean: 


38600.57142857143

In [67]:
ratio_c4 = pop_c4 / rest_c4
ratio_c4

1678.2857142857142

### Cluster 5

In [26]:
cluster5 = analysis_df.loc[analysis_df['Cluster Labels'] == 4]
cluster5

,ZIP Code,Restaurants Count,Population,Cluster Labels
7,94110,23,69333,4
9,94112,25,79407,4


In [56]:
print('Restaurants Count mean: ')
rest_c5 = cluster5.loc[:,'Restaurants Count'].mean()
rest_c5

Restaurants Count mean: 


24.0

In [57]:
print('Population mean: ')
pop_c5 = cluster5.loc[:,'Population'].mean()
pop_c5

Population mean: 


74370.0

In [68]:
ratio_c5 = pop_c5 / rest_c5
ratio_c5

3098.75

## 4. Result summary and Conclusion

In order to summarize and discuss the results, we will put together the mean values of each cluster in a new summary dataframe.

### Summary

In [70]:
summary = pd.DataFrame(columns=['Cluster', 'Restaurants Count mean', 'Population mean', 'Ratio'])
summary['Cluster']=[1, 2, 3, 4, 5]
summary['Restaurants Count mean'] = [rest_c1, rest_c2, rest_c3, rest_c4, rest_c5]
summary['Population mean'] = [pop_c1, pop_c2, pop_c3, pop_c4, pop_c5]
summary['Ratio'] = [ratio_c1, ratio_c2, ratio_c3, ratio_c4, ratio_c5]
summary.set_index('Cluster')

,Restaurants Count mean,Population mean,Ratio
Cluster,,,
1,20.125,26565.375000,1320.018634
2,19.000,4339.125000,228.375000
3,28.000,56003.500000,2000.125000
4,23.000,38600.571429,1678.285714
5,24.000,74370.000000,3098.750000


### Discussion

The best option for this business case is to open the first restaurant in **Cluster 5**, as it presents the **highest *Population vs Restaurants ratio***. This cluster presents, by far, the highest population density, and it is not the cluster with the highest number of restaurants, which justifies the calculated ratio and the **decision to open the restaurant in either ZIP Code 94110 or 94112**.

In [71]:
cluster5

,ZIP Code,Restaurants Count,Population,Cluster Labels
7,94110,23,69333,4
9,94112,25,79407,4


# Thank you for reviewing my Capstone Project!